# MBT v.01

### Recebendo dados do usuário

In [22]:
import sympy as sp
sp.init_printing()

l_viga = float(input('Qual o comprimento da viga em mm:  '))
h_viga = float(input('Qual a altura da viga em mm:  '))
rec = float(input('Qual o recobrimento:  '))
lh = l_viga / h_viga
print(f'O L/h da viga é: {lh}')

# Zona morta para vigas parede com L/h < 1.00
if lh <= 1.00:
    lh = 1.00001

# Ângulo de formação da Biela

ang_biela = -15.2 * sp.ln(lh) + 66.281  # Mapeamento Ansys
print(f'o Ângulo entre as barras é: {ang_biela}°')

# Dimensões da treliça

ang_biela *= (3.1415926535897932384626433832795/180.)
l_bzinf = l_viga
z = 0
if lh >= 2:
    z = 0.90 * (h_viga - (rec + 6.250))
if lh < 2:
    pdz = -0.1259 * (lh ** 2) + 0.6107 * lh + 0.1939  # Mapeamento Ansys correlacionando com a altura útil
    z = pdz * (h_viga - (rec + 6.250))

print(f'O valor do braço de alavanca (Z) é {z} mm')
l_biela = z / sp.sin(ang_biela)
l_bzsup = l_bzinf - 2 * (z / sp.tan(ang_biela))

print(f'O comprimento das bielas comprimidas é: {l_biela} mm')
print(f'O comprimento do banzo superior é: {l_bzsup} mm')
print(f'O comprimento do banzo inferior é: {l_bzinf} mm')

Qual o comprimento da viga em mm:  5000
Qual a altura da viga em mm:  500
Qual o recobrimento:  25
O L/h da viga é: 10.0
o Ângulo entre as barras é: 31.2817065864905°
O valor do braço de alavanca (Z) é 421.875 mm
O comprimento das bielas comprimidas é: 812.475739359354 mm
O comprimento do banzo superior é: 3611.27639694210 mm
O comprimento do banzo inferior é: 5000.0 mm


### Passo 1: Montagem das matrizes e vetores

In [23]:
# Criando as constantes em variáveis simbólicas

A_12, A_23, A_34, A_41 = sp.symbols(['A_12', 'A_23', 'A_34', 'A_41'])
I_12, I_23, I_34, I_41 = sp.symbols(['I_12', 'I_23', 'I_34', 'I_41'])
E_12, E_23, E_34, E_41 = sp.symbols(['E_12', 'E_23', 'E_34', 'E_41'])
L_12, L_23, L_34, L_41 = sp.symbols(['L_12', 'L_23', 'L_34', 'L_41'])
P_11, P_12, P_21, P_22, P_31, P_32, P_41, P_42 = sp.symbols(['P_11', 'P_12', 'P_21', 'P_22', 'P_31', 'P_32', 'P_41', 'P_42'])

# Definindo as variáveis dos ângulos

alpha = sp.symbols('alpha')
c = sp.cos(alpha)
s = sp.sin(alpha)

#### a) Matriz de Equiíbrio - L

In [24]:
# Matriz de Equilibrio - L

Lo_eq = sp.Matrix([[-c,  s/L_12,-s/L_12,  0,      0,      0,   0,      0,      0,    0,      0,      0],
                   [-s, -c/L_12, c/L_12,  0,      0,      0,   0,      0,      0,    0,      0,      0],
                   [ 0,       1,      0,  0,      0,      0,   0,      0,      0,    0,      0,      0],
                   [ c, -s/L_12, s/L_12, -c, s/L_23,-s/L_23,   0,      0,      0,    0,      0,      0],
                   [ s,  c/L_12,-c/L_12, -s,-c/L_23, c/L_23,   0,      0,      0,    0,      0,      0],
                   [ 0,       0,     -1,  0,      1,      0,   0,      0,      0,    0,      0,      0],
                   [ 0,       0,      0,  c,-s/L_23, s/L_23,  -c, s/L_34,-s/L_34,    0,      0,      0],
                   [ 0,       0,      0,  s, c/L_23,-c/L_23,  -s,-c/L_34, c/L_34,    0,      0,      0],
                   [ 0,       0,      0,  0,      0,     -1,   0,      1,      0,    0,      0,      0],
                   [ 0,       0,      0,  0,      0,      0,   c,-s/L_34, s/L_34,   -c, s/L_41,-s/L_41],
                   [ 0,       0,      0,  0,      0,      0,   s, c/L_34,-c/L_34,   -s,-c/L_41, c/L_41],
                   [ 0,       0,      0,  0,      0,      0,   0,      0,     -1,    0,      1,      0],
                   [ 0,       0,      0,  0,      0,      0,   0,      0,      0,    c,-s/L_41, s/L_41],
                   [ 0,       0,      0,  0,      0,      0,   0,      0,      0,    s, c/L_41,-c/L_41],
                   [ 0,       0,      0,  0,      0,      0,   0,      0,      0,    0,      0,    -1]])




In [25]:
# Colocando as restrições de apoio

L_eq = Lo_eq.copy()
L_eq.row_del(0)
L_eq.row_del(0)
L_eq.row_del(2)
L_eq

⎡  0        1        0        0        0         0         0        0         
⎢                                                                             
⎢        -sin(α)   sin(α)            sin(α)   -sin(α)                         
⎢cos(α)  ────────  ──────  -cos(α)   ──────   ────────     0        0         
⎢          L₁₂      L₁₂               L₂₃       L₂₃                           
⎢                                                                             
⎢  0        0        -1       0        1         0         0        0         
⎢                                                                             
⎢                                   -sin(α)    sin(α)             sin(α)   -si
⎢  0        0        0     cos(α)   ────────   ──────   -cos(α)   ──────   ───
⎢                                     L₂₃       L₂₃                L₃₄       L
⎢                                                                             
⎢                                    cos(α)   -cos(α

In [26]:
# Matriz de Equilirio Transposta

L_eqt = Lo_eq.copy()
L_eqt = L_eqt.T
L_eqt.col_del(0)
L_eqt.col_del(0)
L_eqt.col_del(2)
L_eqt

⎡0   cos(α)   0      0         0      0      0         0      0      0        
⎢                                                                             
⎢   -sin(α)                                                                   
⎢1  ────────  0      0         0      0      0         0      0      0        
⎢     L₁₂                                                                     
⎢                                                                             
⎢    sin(α)                                                                   
⎢0   ──────   -1     0         0      0      0         0      0      0        
⎢     L₁₂                                                                     
⎢                                                                             
⎢0  -cos(α)   0    cos(α)    sin(α)   0      0         0      0      0        
⎢                                                                             
⎢    sin(α)       -sin(α)    cos(α)                 

#### b) Matriz de Rigidez do Elemento - k

In [27]:
# Matriz de Rigidez do Elemento - k

k_el = sp.Matrix([[      (3*P_11/(4-P_11*P_12))*(4*E_12*I_12/L_12),(3*P_11*P_12/(4-P_11*P_12))*(-2*E_12*I_12/L_12),        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [(3*P_11*P_12/(4-P_11*P_12))*(-2*E_12*I_12/L_12),      (3*P_11/(4-P_11*P_12))*(4*E_12*I_12/L_12),        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [                                        0,                                        0, E_12*A_12/L_12, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [                                        0,                                        0,            0,      (3*P_21/(4-P_21*P_22))*(4*E_23*I_23/L_23),(3*P_21*P_22/(4-P_21*P_22))*(-2*E_23*I_23/L_23),        0,  0, 0, 0, 0, 0, 0],
                  [                                        0,                                        0,            0,(3*P_21*P_22/(4-P_21*P_22))*(-2*E_23*I_23/L_23),      (3*P_21/(4-P_21*P_22))*(4*E_23*I_23/L_23),        0,  0, 0, 0, 0, 0, 0],
                  [                                        0,                                        0,            0,                                        0,                                        0, E_23*A_23/L_23,  0, 0, 0, 0, 0, 0],
                  [                                        0,                                        0,            0,                                        0,                                        0,            0,      (3*P_31/(4-P_31*P_32))*(4*E_34*I_34/L_34),(3*P_31*P_32/(4-P_31*P_32))*(-2*E_34*I_34/L_34),        0,  0,  0,  0],
                  [                                        0,                                        0,            0,                                        0,                                        0,            0,(3*P_31*P_32/(4-P_31*P_32))*(-2*E_34*I_34/L_34),      (3*P_31/(4-P_31*P_32))*(4*E_34*I_34/L_34),        0,  0,  0,  0],
                  [                                        0,                                        0,            0,                                        0,                                        0,            0,                                        0,                                        0, E_34*A_34/L_34,  0,  0,  0],
                  [                                        0,                                        0,            0,                                        0,                                        0,            0,                                        0,                                        0,        0,      (3*P_41/(4-P_41*P_42))*(4*E_41*I_41/L_41),(3*P_41*P_42/(4-P_41*P_42))*(-2*E_41*I_41/L_41),        0],
                  [                                        0,                                        0,            0,                                        0,                                        0,            0,                                        0,                                        0,        0,(3*P_41*P_42/(4-P_41*P_42))*(-2*E_41*I_41/L_41),      (3*P_41/(4-P_41*P_42))*(4*E_41*I_41/L_41),        0],
                  [                                        0,                                        0,            0,                                        0,                                        0,            0,                                        0,                                        0,        0,                                        0,                                        0, E_41*A_41/L_41]])

k_el

⎡  12⋅E₁₂⋅I₁₂⋅P₁₁     -6⋅E₁₂⋅I₁₂⋅P₁₁⋅P₁₂                                      
⎢──────────────────   ───────────────────     0              0                
⎢L₁₂⋅(-P₁₁⋅P₁₂ + 4)    L₁₂⋅(-P₁₁⋅P₁₂ + 4)                                     
⎢                                                                             
⎢-6⋅E₁₂⋅I₁₂⋅P₁₁⋅P₁₂     12⋅E₁₂⋅I₁₂⋅P₁₁                                        
⎢───────────────────  ──────────────────      0              0                
⎢ L₁₂⋅(-P₁₁⋅P₁₂ + 4)  L₁₂⋅(-P₁₁⋅P₁₂ + 4)                                      
⎢                                                                             
⎢                                          A₁₂⋅E₁₂                            
⎢         0                    0           ───────           0                
⎢                                            L₁₂                              
⎢                                                                             
⎢                                                   

#### c) Vetor das Cargas Nodais (Cargas pontuais e Cargas Momentos) - Lambda

In [28]:
# Criando o vetor das cargas nodais

# F8 = 10kN
# F11 = 10kN

F1, F2, F3, F4, F5, F6, F7, F8, F9, F10, F11, F12 = sp.symbols(['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12'])

F = sp.Matrix([F1, F2, F3, F4, F5, F6, F7, F8, F9, F10, F11, F12])

In [29]:
# Aplicando a restrição dos apoios
Fn = F.copy()
Fn.row_del(0)
Fn.row_del(0)
Fn.row_del(2)
Fn


⎡F₃ ⎤
⎢   ⎥
⎢F₄ ⎥
⎢   ⎥
⎢F₆ ⎥
⎢   ⎥
⎢F₇ ⎥
⎢   ⎥
⎢F₈ ⎥
⎢   ⎥
⎢F₉ ⎥
⎢   ⎥
⎢F₁₀⎥
⎢   ⎥
⎢F₁₁⎥
⎢   ⎥
⎣F₁₂⎦

#### d) Vetor dos Deslocamentos Nodais (Deslocamentos Lineares e Rotacionais) - Delta

In [30]:
# Criando o vetor dos deslocamentos nodais
delta = sp.symbols('delta')
delta1, delta2, delta3, delta4, delta5, delta6, delta7, delta8, delta9, delta10, delta11, delta12 = sp.symbols(['delta1', 'delta2', 'delta3', 'delta4', 'delta5', 'delta6', 'delta7', 'delta8', 'delta9', 'delta10', 'delta11', 'delta12'])

dt = sp.Matrix([delta1, delta2, delta3, delta4, delta5, delta6, delta7, delta8, delta9, delta10, delta11, delta12])


In [31]:
# Aplicando a restrição dos apoios
dlt = dt.copy()
dlt.row_del(0)
dlt.row_del(0)
dlt.row_del(2)
dlt

⎡δ₃ ⎤
⎢   ⎥
⎢δ₄ ⎥
⎢   ⎥
⎢δ₆ ⎥
⎢   ⎥
⎢δ₇ ⎥
⎢   ⎥
⎢δ₈ ⎥
⎢   ⎥
⎢δ₉ ⎥
⎢   ⎥
⎢δ₁₀⎥
⎢   ⎥
⎢δ₁₁⎥
⎢   ⎥
⎣δ₁₂⎦

#### e) Vetor das Deformações Correspondentes (Deformações Lineares e Angulares) - Theta

In [32]:
# Criando o Vetor das Deformações Correspondentes
theta = sp.symbols('theta')
theta1, theta2, theta3, theta4, theta5, theta6, theta7, theta8, theta9, theta10, theta11, theta12 = sp.symbols(['theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7', 'theta8', 'theta9', 'theta10', 'theta11', 'theta12'])
teta = sp.Matrix([theta1, theta2, theta3, theta4, theta5, theta6, theta7, theta8, theta9, theta10, theta11, theta12])
teta

⎡θ₁ ⎤
⎢   ⎥
⎢θ₂ ⎥
⎢   ⎥
⎢θ₃ ⎥
⎢   ⎥
⎢θ₄ ⎥
⎢   ⎥
⎢θ₅ ⎥
⎢   ⎥
⎢θ₆ ⎥
⎢   ⎥
⎢θ₇ ⎥
⎢   ⎥
⎢θ₈ ⎥
⎢   ⎥
⎢θ₉ ⎥
⎢   ⎥
⎢θ₁₀⎥
⎢   ⎥
⎢θ₁₁⎥
⎢   ⎥
⎣θ₁₂⎦

#### f) Vetor dos Esforços Seccionais Internos (Momentos Fletores e Esforços Normais) - m

In [33]:
# Vetor dos Esforços Seccionais Internos
m1, m2, n1, m3, m4, n2, m5, m6, n3, m7, m8, n4 = sp.symbols(['m1', 'm2', 'n1', 'm3', 'm4', 'n2', 'm5', 'm6', 'n3', 'm7', 'm8', 'n4'])
m = sp.Matrix([m1, m2, n1, m3, m4, n2, m5, m6, n3, m7, m8, n4])
m


⎡m₁⎤
⎢  ⎥
⎢m₂⎥
⎢  ⎥
⎢n₁⎥
⎢  ⎥
⎢m₃⎥
⎢  ⎥
⎢m₄⎥
⎢  ⎥
⎢n₂⎥
⎢  ⎥
⎢m₅⎥
⎢  ⎥
⎢m₆⎥
⎢  ⎥
⎢n₃⎥
⎢  ⎥
⎢m₇⎥
⎢  ⎥
⎢m₈⎥
⎢  ⎥
⎣n₄⎦

In [34]:
K_global = L_eq * k_el * L_eqt